In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from keras.layers import Dense,Dropout
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso,Ridge
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.layers.advanced_activations import ReLU, PReLU
from keras.optimizers import SGD, Adam
from scipy.stats import mode

In [30]:
train_x = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\dataset\\train_x_xgboost.csv')
train_y = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\dataset\\train_y_xgboost.csv')
test_x = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\dataset\\test_x_xgboost.csv')

In [31]:
train_x_nn = pd.read_csv('.//dataset//train_x_NN.csv')
train_y_nn = pd.read_csv('.//dataset//train_y_NN.csv')
test_x_nn = pd.read_csv('.//dataset//test_x_NN.csv')

In [32]:
from models import Model1xgb,Model1xgb2, Model1NNproba,Model1NN2proba,Model1ramdom,Model2KMeans,Model2KNN,Model3logistic

In [33]:
def predict_cv_classfier(model,train_x, train_y, test_x):
    

    preds = []
    preds_test = []
    va_idxes = []


    kf = KFold(n_splits=4, shuffle=True, random_state= 71)
    for i , (tr_idx, va_idx) in enumerate (kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    tmp = np.stack(preds_test, axis =1)
    mode_test, mode_counts = mode(tmp, axis=1)

    preds_test = mode_test
    preds_size = preds_test.shape[0]
    preds_test = preds_test.reshape(preds_size,)
    
    return pred_train, preds_test

In [34]:
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []
    
    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    for i , (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)
        
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds)
    order = np.argsort(va_idxes)
    pred_train = preds[order]
    
    preds_test = np.mean(preds_test, axis=0)
    
    return pred_train, preds_test

In [13]:
model1_a = Model2KNN()
pred_train_1a , preds_test_1a = predict_cv_classfier(model1_a, train_x, train_y, test_x)

model1_b = Model2KMeans()
pred_train_1b, preds_test_1b = predict_cv_classfier(model1_b, train_x, train_y, test_x)

print(f'logloss: {log_loss(train_y, pred_train_1a, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1b, eps=1e-7):.4f}')

C:\Users\odoru\SIGNATE_time_deposit_account\models.py:271: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.model.fit(tr_x, tr_y)
C:\Users\odoru\SIGNATE_time_deposit_account\models.py:271: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.model.fit(tr_x, tr_y)
C:\Users\odoru\SIGNATE_time_deposit_account\models.py:271: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.model.fit(tr_x, tr_y)
C:\Users\odoru\SIGNATE_time_deposit_account\models.py:271: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.model.fit(tr_x, tr_y)


logloss: 1.2561
logloss: 6.7286


In [ ]:
,Model1NN2proba,Model1ramdom,Model2KMeans,Model2KNN,Model3logistic

In [27]:
model_1c = Model1xgb()
pred_train_1c, pred_test_1c = predict_cv(model_1c, train_x, train_y, test_x)

model_1d = Model1NNproba()
pred_train_1d, pred_test_1d = predict_cv(model_1d, train_x_nn, train_y, test_x_nn)


model_1e = Model1ramdom()
pred_train_1e, pred_test_1e = predict_cv(model_1e, train_x, train_y, test_x)


model_1f = Model1xgb2()
pred_train_1f, pred_test_1f = predict_cv(model_1f, train_x, train_y, test_x)

model_1g = Model1NN2proba()
pred_train_1g, pred_test_1g = predict_cv(model_1e, train_x_nn, train_y, test_x_nn)


[13:38:55] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-error:0.06799	eval-error:0.07055
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 20 rounds.
[1]	train-error:0.06824	eval-error:0.07041
[2]	train-error:0.06760	eval-error:0.07070
[3]	train-error:0.06819	eval-error:0.07026
[4]	train-error:0.06716	eval-error:0.07011
[5]	train-error:0.06770	eval-error:0.06922
[6]	train-error:0.06731	eval-error:0.06937
[7]	train-error:0.06750	eval-error:0.06937
[8]	train-error:0.06741	eval-error:0.06967
[9]	train-error:0.06735	eval-error:0.06922
[10]	train-error:0.06701	eval-error:0.068

[22]	train-error:0.06622	eval-error:0.07100
[23]	train-error:0.06622	eval-error:0.07100
[24]	train-error:0.06632	eval-error:0.07055
[25]	train-error:0.06637	eval-error:0.07041
[26]	train-error:0.06627	eval-error:0.07070
[27]	train-error:0.06617	eval-error:0.07026
[28]	train-error:0.06632	eval-error:0.07055
[29]	train-error:0.06608	eval-error:0.07070
[30]	train-error:0.06593	eval-error:0.07055
[31]	train-error:0.06568	eval-error:0.07041
[32]	train-error:0.06553	eval-error:0.07085
[33]	train-error:0.06534	eval-error:0.07070
[34]	train-error:0.06519	eval-error:0.07026
[35]	train-error:0.06495	eval-error:0.07070
[36]	train-error:0.06499	eval-error:0.07100
[37]	train-error:0.06485	eval-error:0.07070
[38]	train-error:0.06460	eval-error:0.07085
[39]	train-error:0.06431	eval-error:0.07070
[40]	train-error:0.06406	eval-error:0.07100
[41]	train-error:0.06391	eval-error:0.07100
[42]	train-error:0.06376	eval-error:0.07085
[43]	train-error:0.06376	eval-error:0.07085
[44]	train-error:0.06342	eval-er

KeyboardInterrupt: 

In [35]:
model_1d = Model1NNproba()
pred_train_1d, pred_test_1d = predict_cv(model_1d, train_x_nn, train_y, test_x_nn)
print(f'logloss: {log_loss(train_y, pred_train_1d, eps=1e-7):.4f}')

Epoch 1/100
159/159 [==============================] - 1s 5ms/step - loss: 0.2447 - accuracy: 0.9237 - val_loss: 0.2122 - val_accuracy: 0.9315
Epoch 2/100
159/159 [==============================] - 1s 4ms/step - loss: 0.2158 - accuracy: 0.9292 - val_loss: 0.2097 - val_accuracy: 0.9330
Epoch 3/100
159/159 [==============================] - 1s 4ms/step - loss: 0.2130 - accuracy: 0.9298 - val_loss: 0.2125 - val_accuracy: 0.9315
Epoch 4/100
159/159 [==============================] - 1s 4ms/step - loss: 0.2079 - accuracy: 0.9309 - val_loss: 0.2097 - val_accuracy: 0.9323
Epoch 5/100
159/159 [==============================] - 1s 4ms/step - loss: 0.2038 - accuracy: 0.9320 - val_loss: 0.2077 - val_accuracy: 0.9317
Epoch 6/100
159/159 [==============================] - 1s 4ms/step - loss: 0.2053 - accuracy: 0.9319 - val_loss: 0.2083 - val_accuracy: 0.9327
Epoch 7/100
159/159 [==============================] - 1s 4ms/step - loss: 0.2000 - accuracy: 0.9330 - val_loss: 0.2085 - val_accuracy: 0.9334

159/159 [==============================] - 1s 4ms/step - loss: 0.1918 - accuracy: 0.9354 - val_loss: 0.2223 - val_accuracy: 0.9305
Epoch 11/100
159/159 [==============================] - 1s 4ms/step - loss: 0.1899 - accuracy: 0.9360 - val_loss: 0.2253 - val_accuracy: 0.9294
Epoch 12/100
159/159 [==============================] - 1s 4ms/step - loss: 0.1887 - accuracy: 0.9362 - val_loss: 0.2216 - val_accuracy: 0.9283
Epoch 13/100
159/159 [==============================] - 1s 4ms/step - loss: 0.1859 - accuracy: 0.9366 - val_loss: 0.2244 - val_accuracy: 0.9293
Epoch 14/100
159/159 [==============================] - 1s 4ms/step - loss: 0.1823 - accuracy: 0.9377 - val_loss: 0.2237 - val_accuracy: 0.9294
Epoch 15/100
159/159 [==============================] - 1s 4ms/step - loss: 0.1819 - accuracy: 0.9383 - val_loss: 0.2272 - val_accuracy: 0.9287
Epoch 16/100
159/159 [==============================] - 1s 4ms/step - loss: 0.1799 - accuracy: 0.9391 - val_loss: 0.2272 - val_accuracy: 0.9286
Epoch

In [18]:
print(f'logloss: {log_loss(train_y, pred_train_1c, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1d, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1e, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1f, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1g, eps=1e-7):.4f}')

logloss: 0.2068
logloss: 1.2561
logloss: 0.2492
logloss: 0.2129
logloss: 0.2545


In [19]:
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b, 'pred_1c':pred_train_1c,'pred_1d':pred_train_1d, 'pred_1e':pred_train_1e, 'pred_1f': pred_train_1f,'pred_1g':pred_train_1g})
test_x_2 = pd.DataFrame({'pred_1a': pred_test_1a, 'pred_1b': pred_test_1b, 'pred_1c': pred_test_1c,'pred_1d':pred_test_1d, 'pred_1e':pred_test_1e, 'pred_1f': pred_test_1f,'pred_1g':pred_test_1g})

NameError: name 'pred_test_1a' is not defined